# 电商产品评论分析

电子商务的发展，极大的拉近了商家和客户之间的距离，随着同质化竞争越来越严重，商家必须能够了解客户的心声，才能对市场的快速变化做出响应。

其中非常重要的方式就是对消费者的文本评论数据进行挖掘分析。从中得到的信息，会非常有利于商品生产厂家自身竞争力的提升。

针对电商产品评论的分析，通过回答以下几个问题：
1. 分析产品与竞争对手产品的用户情感倾向
2. 从评论文本中挖掘产品的优点和不足
3. 提炼竞争对手产品的卖点和研发的趋势

## 1. 文本数据处理

评论数据的获取通常需要爬虫从电商网站或论坛采集，在这里假设数据已经采集成功，首先来从原始数据中查看美的相关的评论。

In [1]:
import pandas as pd

input_file = '../data/huizong.csv.tgz'
data = pd.read_csv(input_file, compression='gzip', encoding='utf-8')
data = data[[u'评论']][data[u'品牌']==u'美的']
data.head(5)

/Users/yingrui/miniconda3/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,评论
95898,京东商城信得过，买的放心，用的省心、安心、放心！
95899,给公司宿舍买的，上门安装很快，快递也送的及时，不错的。给五分吧
95900,美的值得信赖，质量不错
95901,不错不错的哦，第一次在京东买这些产品，感觉相当好
95902,很满意，水方一晚上都还是热的早上还能再洗，


读取到美的的产品评论之后，首先会去除掉数据中大量价值含量很低甚至没有价值含量的条目。如果将这些数据引入进行后续的词频统计乃至情感分析，会对分析造成很大的影响，得到的结果的质量也必然存在问题。

在利用这些文本数据开始分析之前，需要对文本进行预处理。文本评论数据的预处理主要由3个部分组成：去除重复、去除机械重复用词、短句删除三种。

### 1.1. 去除重复
#### 1.1.1. 评论数据重复的原因
1. 电商平台为了避免一些客户长时间不进行评论，往往会在超出规定的时间之后，自动替用户做出评论（国美），而这种评论是没有意义的。
2. 同一个人可能会出现重复评论，因为同一个人可能会购买多款产品，然后在评论过程中为了省事，提交了同样的评论。
3. 不同人认真提交的评论往往不会相同，因为会从不同的视角来评价产品。大量内容完全相同的评论，可能是直接复制，粘贴的评论，相同的评论只有最早的一条可能才有意义。

#### 1.1.2. 常见去重算法
1. 编辑距离
2. SimHash
3. 比较去重

通常采用编辑距离和SimHash去重，也会引入一些问题，例如会删除掉一些非常类似，但是意义完全不同的评论。所以在短文本中，也就是评论数据中，最终选择了去掉完全相同的评论。

In [2]:
output_file = './tmp/meidi_jd_process_1.txt'
from os import makedirs
makedirs('./tmp', exist_ok=True)

total_comments = len(data)
comments = pd.DataFrame(data[u'评论'].unique())
total_unique_comments = len(comments)

print('删除了%d条评论' % (total_comments - total_unique_comments))
comments.to_csv(output_file, index = False, header = False, encoding = 'utf-8')

删除了2725条评论


### 1.2. 去除机械重复
#### 1.2.1. 机械重复用词的现象
由于电商品牌的文本评论数据质量参差不齐，没有意义的文本数据很多。有很多评论是连续重复的语料，例如：
* 非常好非常好非常好非常好非常好非常好非常好非常好
* 好呀好呀好呀好呀好呀好呀好呀好呀好呀
这一类是存在连续重复的语料，也是最常见的无意义语料。

#### 1.2.2. 机械重复去词的规则说明
机械重复的判断可以通过建立两个存放字符的列表来完成，通过扫描整个字符串，查找到重复出现的词或短句，然后删除。查找过程如下：
1. 逐字扫描输入句子，将字符添加到第一个列表
2. 如果出现一个字符和第一个个列表的首字相同，则添加到第二个列表
3. 继续逐字扫描输入句子，将字符添加到第二个列表
4. 如果第二个列表和第一个列表相同，则找到了重复句子
5. 如果第二个列表和第一个列表不同，则清空两个列表
6. 返回1，从下一个位置开始扫描

### 1.3. 短句删除

#### 1.3.1. 短句删除的原因
去除机械重复词之后，可能会生成很多短句，或者本来就很短的句子。例如：很不错、好、质量差等。因为词越少，能够表达的意思就相应的减少。过短的评论通常是没有意义的，也应考虑是否删除，以及如何删除。通常选择一个长度来决定，小于这个长度的短句都会被删除。

## 2. 情感分析

### 2.1.1. 基于神经网络的情感分析方法
1. Stanford NLP深度学习情感分析，是将句子构建成一个树状结构之后，再进行情感倾向性分类的方法。
2. Fasttext基于词向量的序列标注算法，对每一个词进行情感倾向标注
3. 文本分类：可以通过训练文本分类器来进行情感倾向性判断。可以尝试用CNN的方法来进行，Stanford NLP也有文本Classifier可以使用。

### 2.1.2. 调用SnowNLP对文本进行情感分析
以下是调用snownlp的例子代码
```python
from snownlp import SnowNLP
s = SnowNLP(u'这个东西真心很赞')
s.words         # [u'这个', u'东西', u'真心',
                #  u'很', u'赞']
s.tags          # [(u'这个', u'r'), (u'东西', u'n'),
                #  (u'真心', u'd'), (u'很', u'd'),
                #  (u'赞', u'Vg')]
s.sentiments    # 0.9769663402895832 positive的概率
```

In [3]:
from snownlp import SnowNLP

input_file = './tmp/meidi_jd_process_1.txt'
comments = pd.read_csv(input_file, encoding = 'utf-8', header = None)
comments = comments[:100] # 仅取100条评论来演示

sentiments = comments[0].apply(lambda comment: SnowNLP(comment).sentiments)

In [ ]:
comments_sentiments = pd.concat([comments, sentiments], axis = 1)
comments_sentiments.head()

,0,0
0,京东商城信得过，买的放心，用的省心、安心、放心！,0.982791
1,给公司宿舍买的，上门安装很快，快递也送的及时，不错的。给五分吧,0.839830
2,美的值得信赖，质量不错,0.999553
3,不错不错的哦，第一次在京东买这些产品，感觉相当好,0.989972
4,很满意，水方一晚上都还是热的早上还能再洗，,0.126814


SnowNLP仅是对数据进行简单的情感分析，所观察到的效果并不是很好，实际使用的时候，需要结合实际的情况，来考虑是否可以直接使用？还是需要自己训练一个模型？

## 3. 主题分析

主题模型在机器学习和自然语言处理领域是用来在一系列文档中发现抽象主题的一种统计模型。如果一篇文档有多个主题，则一些特定的可代表不同主题的词语会反复出现，此时，运用主题模型，能够发现文本中使用词语的规律，并且把规律相似的文本联系到一起，以寻求非结构化的文档集中的有用信息。

例如：针对热水器的商品评论，代表热水器特征的词语，如：安装、出水量、服务等会频繁的出现在评论中，运用主题模型，将与热水器代表性特征相关的情感描述性词语，同相应的特征词语联系起来，从而深入了解热水器评价的聚焦点及用户对于某一特征的情感倾向。

### 3.1. LDA主题模型
LDA模型作为其中一种主题模型，属于无监督的生成式主题概率模型。LDA认为每一篇文档的每一个词都是通过“一定的概率选择了某个主题，并从这个主题中以一定的概率选择了某个词语。”

LDA模型也被称为3层贝叶斯概率模型，包含文档（d）、主题（z）和词（w）3层结构。通过LDA主题模型，能够挖掘数据集内部的潜在主题，进而分析数据集的关注点及其相关特征词。

In [ ]:
#参数初始化
negfile = '../data/meidi_jd_neg_cut.txt.tgz'
posfile = '../data/meidi_jd_pos_cut.txt.tgz'
stoplist = '../data/stoplist.txt'

neg = pd.read_csv(negfile, compression='gzip', encoding = 'utf-8', header = None) #读入数据
pos = pd.read_csv(posfile, compression='gzip', encoding = 'utf-8', header = None)
stop = pd.read_csv(stoplist, encoding = 'utf-8', header = None, sep = 'tipdm')
stop = [' ', ''] + list(stop[0]) #Pandas自动过滤了空格符，这里手动添加

neg[1] = neg[0].astype(str).apply(lambda s: s.split(' ')) #定义一个分割函数，然后用apply广播
neg[2] = neg[1].apply(lambda x: [i for i in x if i not in stop]) #逐词判断是否停用词，思路同上
pos[1] = pos[0].astype(str).apply(lambda s: s.split(' '))
pos[2] = pos[1].apply(lambda x: [i for i in x if i not in stop])

from gensim import corpora, models

#负面主题分析 
neg_dict = corpora.Dictionary(neg[2]) #建立词典
neg_corpus = [neg_dict.doc2bow(i) for i in neg[2]] #建立语料库
neg_lda = models.LdaModel(neg_corpus, num_topics = 3, id2word = neg_dict) #LDA模型训练

#正面主题分析
pos_dict = corpora.Dictionary(pos[2])
pos_corpus = [pos_dict.doc2bow(i) for i in pos[2]]
pos_lda = models.LdaModel(pos_corpus, num_topics = 3, id2word = pos_dict)


/Users/yingrui/miniconda3/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
for i in range(3):
  print(neg_lda.print_topic(i)) #输出每个主题

for i in range(3):
  print(pos_lda.print_topic(i)) #输出每个主题